In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [32]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 20
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [33]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.NN as nn

class RNN(nn.NN):

    def __init__(self, D, H, char2idx, idx2char):
        self.D = D
        self.H = H
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        super().__init__(D, D, H, None, None, loss='cross_ent', nonlin='relu')

    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, train=True):
        Wxh, Whh, Why = self.model['Wxh'], self.model['Whh'], self.model['Why']
        bh, by = self.model['bh'], self.model['by']

        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        hprev = h.copy()

        h, h_cache = l.tanh_forward(X_one_hot @ Wxh + hprev @ Whh + bh)
        y, y_cache = l.fc_forward(h, Why, by)

        cache = (X_one_hot, Whh, h, hprev, y, h_cache, y_cache)

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, dy, dh_next, cache):
        X, Whh, h, hprev, y, h_cache, y_cache = cache

        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dh = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dh
        dWhh = hprev.T @ dh
        dWxh = X.T @ dh
        dh_next = dh @ Whh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)

        return grad, dh_next

    def train_step_fwd(self, X_train, y_train, h):

        # Forward 1st layer
        ys, caches = [], []
        for x in X_train:
            y_pred, h, cache = self.forward(x, h, train=True) # self.model[0] for the 1st layer
            ys.append(y_pred)
            caches.append(cache)
        
        return ys, caches

    def train_step_bwd(self, X_train, y_train, ys, caches):

        # loss/error function: last layer
        loss, dys = 0.0, []
        for y_pred, y in zip(ys, y_train):
            loss += loss_fun.cross_entropy(self.model, y_pred, y, lam=0)/ y_train.shape[0]
            dy = loss_fun.dcross_entropy(y_pred, y)
            dys.append(dy)

        # Backward 1st layer
        dh_next = np.zeros((1, self.H))
        grads = {k: np.zeros_like(v) for k, v in self.model.items()}

        for t in reversed(range(len(ys))):
            grad, dh_next = self.backward(dys[t], dh_next, caches[t]) # self.model[0] for the 1st layer
            for k in grads.keys():
                grads[k] += grad[k]

        # Clipping grads for exploding grad problems
        for k, v in grads.items():
            grads[k] = np.clip(v, -5., 5.)

        return grads, loss

    def test_step_fwd(self, X_test_seed, h):
        chars = [self.idx2char[X_test_seed]]
        idx_list = list(range(self.vocab_size))
        x = X_test_seed
        size = 100

        # last layer: train=False
        for t in range(size-1):
            y_pred, h, cache = self.forward(x, h, train=False)
            idx = np.random.choice(idx_list, p=y_pred.ravel())
            chars.append(self.idx2char[idx])
            x = idx
        
        return ''.join(chars)


    def _init_model(self, D, C, H):
        self.model = dict(
            Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
            Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
            Why=np.random.randn(H, D) / np.sqrt(C / 2.),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

In [34]:
net = RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [35]:
import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    M = {k: np.zeros_like(v) for k, v in nn.model.items()}
    R = {k: np.zeros_like(v) for k, v in nn.model.items()}
    beta1 = .9
    beta2 = .999

    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    idx = 0
    state = nn.initial_state()
    smooth_loss = -np.log(1.0 / len(set(X_train)))

    for iter in range(1, n_iter + 1):
        t = iter

        if idx >= len(minibatches):
            idx = 0
            state = nn.initial_state()

        X_mini, y_mini = minibatches[idx]
        idx += 1

        if iter % print_after == 0:
            print("=========================================================================")
            print('Iter-{} loss: {:.4f}'.format(iter, smooth_loss))
            print("=========================================================================")

            sample = nn.test_step_fwd(X_mini[0], state)
            print(sample)

            print("=========================================================================")
            print()
            print()

        ys, caches = nn.train_step_fwd(X_mini, y_mini, state)
        grad, loss = nn.train_step_bwd(X_mini, y_mini, ys, caches)
        smooth_loss = 0.999 * smooth_loss + 0.001 * loss

        for k in grad:
            M[k] = util.exp_running_avg(M[k], grad[k], beta1)
            R[k] = util.exp_running_avg(R[k], grad[k]**2, beta2)

            m_k_hat = M[k] / (1. - beta1**(t))
            r_k_hat = R[k] / (1. - beta2**(t))

            nn.model[k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

    return nn

In [36]:
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, 
                print_after=print_after)

Iter-1000 loss: 3.4480
izaeBwan nnpcnynlbte  1 tp nl liaa cerfn. ta afis iA ao  ppepin are ptd' Tae anpan lnre .eva an'tI. 


Iter-2000 loss: 2.7291
orJas lotmme Wiula, pproccciayau andirat on thenEe rde turesd mro, the Roptedifnd the wotees optha i


Iter-3000 loss: 2.2372
t-ean onithigs onbeg an cNvpslahes Oinlurgr the tointo edspe4 utir i tiloulcTai. mhonakinciantictori


Iter-4000 loss: 1.9126
xpwrthe wartecttiestita and targext iota o the tometie tiperoum the rourd pin lye sirghe EurerWor h 


Iter-5000 loss: 1.6940
e ouoptrcinory Hhalimes fhe Orit2d meatKof 1m thes xiathe ffanarghkiofan hitheg op tnver Asint. is a


Iter-6000 loss: 1.5310
d in the Pwerd Stree torndrith sukfouldored expo the Eigt e te tirst, the GL0p8685. E pevexbe horld'


Iter-7000 loss: 1.4040
eddely ishan wopilowatc Tpercorico Japenota in horan'ithe oupor-Japen isokexter ern tivol whelineo A


Iter-8000 loss: 1.3002
ed if 18b2 curar-Japan isserghrtokuou Japae worthe Ioferof d is liciditiroE ho move West pirin,

In [ ]:

=========================================================================
Iter-11000 loss: 1.0761
=========================================================================
elior anas uItur whe Eopreg issictha thertaty ss in lapenigon 1947 Uy fedpicccast the werges Sing, t
=========================================================================


=========================================================================
Iter-12000 loss: 1.0250
=========================================================================
d verthec mond cI tevinite ion ou at yntan int rurgh liming sigit8, cc and If is peas ma aedtimy in 
=========================================================================


=========================================================================
Iter-13000 loss: 0.9639
=========================================================================
the worlde. Emgioiko5 Eest of his ralal-lasisy cle whe wotthe iast mu the%e piotceresethasiyiter Sea
=========================================================================


Out[36]:
<__main__.RNN at 0x7fd46e5db898>

In [ ]:
=========================================================================
Iter-11000 loss: 1.0594
=========================================================================
ectsl ingbal 95umboly oporcar ofuthod rlicherevire parint, asestred Sin8. Aaperthe coal os he craies
=========================================================================


=========================================================================
Iter-12000 loss: 1.0032
=========================================================================
d Hifrlalige the Ullet the Glliral geporosth-larod Wan, whe Japan oupest offoporyf ime rart mitity c
=========================================================================


=========================================================================
Iter-13000 loss: 0.9542
=========================================================================
tikicr, Hobons Wheth rire d malarewesc anlobsi. Japandes -laroby tory if bom lrcman "x4s folatatiwed
=========================================================================

In [ ]:
=========================================================================
Iter-11000 loss: 1.0775
=========================================================================
enod frig bylyrboth linty. Fnd ankeItal khaun the Gceutere aat thethe athes unthe orothocoketur prou
=========================================================================


=========================================================================
Iter-12000 loss: 1.0269
=========================================================================
d Japan is  maling ankedg palinusen-Japandsi cturingiss eraltegcors fubolallimg pre tirtes areataly 
=========================================================================


=========================================================================
Iter-13000 loss: 0.9811
=========================================================================
tory 2idicaren . Ampand the kpompof ciandeiggast The celoco hothe Fperth andsthe ongeth cankedich". 
=========================================================================


Out[23]:
<__main__.RNN at 0x7fd46e63cfd0>

In [ ]:
=========================================================================
Iter-1000 loss: 3.4327
=========================================================================
izlt rticha ls por ao ahief iroladn lotansrise bn thss an mimUn; ritys aaa yh and eni oe iau  i a t 
=========================================================================


=========================================================================
Iter-2000 loss: 2.7259
=========================================================================
o1 as the caxnttaidedesturtuinrman thd 10Nm–e meusth ifxthe 8o3ben andith bivithe .oghicn Jare sh wo
=========================================================================


=========================================================================
Iter-3000 loss: 2.2537
=========================================================================
terte the .estloy 7etoo-trodJapan's istercinare atre , phe, chosambon t on eatoreticdsst per on and 
=========================================================================


=========================================================================
Iter-4000 loss: 1.9144
=========================================================================
xt  9upa thethechanSorCathellD本e ctoSeW oned cnhes andthily ikenurinErtu1yku, sangesttemelWert. Tsed
=========================================================================


=========================================================================
Iter-5000 loss: 1.6851
=========================================================================
e l-panglorylid sr iyI5, we couxtst Es lperer Was rhe he thollan-wer terily and s0an ilals, E9glsin 
=========================================================================


=========================================================================
Iter-6000 loss: 1.5071
=========================================================================
d rad onurar meotor a9d rod frurlorpegenceecino i9al and and Rulbod Japand At toughenthoxgentiy ry p
=========================================================================


=========================================================================
Iter-7000 loss: 1.3761
=========================================================================
e tortr.he Asitijy bicoletrighennk instl inch isalerinithr racolylce wouthitary w2slx ec tomir 12. J
=========================================================================


=========================================================================
Iter-8000 loss: 1.2648
=========================================================================
eatured-Lonfto ycyturithes lion ing he aturth tar thith5enfl ofllr'g toreate Nes totateind, inithtin
=========================================================================


=========================================================================
Iter-9000 loss: 1.1783
=========================================================================
caly anmeran-lahe apee wark "nsi".OJSmanu as lalary erean ithe ha"The im the Rass " urthe Gobul:342r
=========================================================================


=========================================================================
Iter-10000 loss: 1.1074
=========================================================================
ed ce womeminakalapane ao Jas nues itst mesto-l honatee worlod ing ha lobate she ned urcanwad canaed
=========================================================================


=========================================================================
Iter-11000 loss: 1.0454
=========================================================================
er of Hemo8ld westh-kaine aintor.k mbst Worlocond, Ahilara with alate bo thevelicity 1par, A9stur ff
=========================================================================


=========================================================================
Iter-12000 loss: 0.9882
=========================================================================
d mivithe worlant mala, fthe worlrata turpe9tiguthe cithe Thichonsl por Huthes atended Toulirit is o
=========================================================================


=========================================================================
Iter-13000 loss: 0.9462
=========================================================================
t oth the Tousolit recalicen ionlroa mevelalour. Japan Easinar ipu fictonasca., hisalede aiof os cen
=========================================================================


Out[15]:
<__main__.RNN at 0x7fd47199e160>

In [ ]:
=========================================================================
Iter-11000 loss: 1.0354
=========================================================================
er asa Rasl of the wors andplons. La.dAsthapirghe Suria samiestar erse gsta ines ur the uregthe fran
=========================================================================


=========================================================================
Iter-12000 loss: 0.9793
=========================================================================
d Astoorchumallos ria mamgeod ofecturen.. Theen woporiveced the iothe, ionhis nhe fith en ins an is 
=========================================================================


=========================================================================
Iter-13000 loss: 0.9217
=========================================================================
tiry co espares iouthe wirlded inp Seakuuitula. Thiand gupit lokeordtid ced pe Unverjl with sisstal 
=========================================================================


Out[151]:
<__main__.RNN at 0x7f00f7961128>

In [ ]:
=========================================================================
Iter-5000 loss: 1.7817
=========================================================================
eThe courd a本 se ove can Slxto aras chhiind ist erear, she a8"- fite prored candinked vintod coban l
=========================================================================


=========================================================================
Iter-6000 loss: 1.6194
=========================================================================
d pirotor af onveu, anthise ap lorog Jatg atilatins is ly wedt on pus xer, thoscoaugest 141 bad ss A
=========================================================================


=========================================================================
Iter-7000 loss: 1.4840
=========================================================================
es Toky, 2ovevongest ar ssururthesiohe Grokthomicctainas orghobaivenslatouthed.difinfe ih Tnviritarr
=========================================================================


=========================================================================
Iter-8000 loss: 1.3743
=========================================================================
e Efr find hicecentinud an the nf urilotpty Niloeo indadkin, wad Ses an imtires, P and Is ionsed S w
=========================================================================


=========================================================================
Iter-9000 loss: 1.2725
=========================================================================
cendise on lasiond isiately of Jaico rlaigith makae wasth. Nsbyo Nhacandtio tomighiv was WarNl poper
=========================================================================


=========================================================================
Iter-10000 loss: 1.1952
=========================================================================
ed GDeloroergucdar himinan Japan waswus he and in 1247. Japan Na foreltbooped in the nalathi an werl
=========================================================================


=========================================================================
Iter-11000 loss: 1.1147
=========================================================================
ed usghin corctorthe Embertithenthg cely bictroled ind Svesea, Oopan sirunite of Nshigestis Pare Chi
=========================================================================


=========================================================================
Iter-12000 loss: 1.0455
=========================================================================
d Chind Ry come hexpandec, peleedinat sed ar aid was onde w rerir dicon topuctlyt maletd purmkcen ch
=========================================================================


=========================================================================
Iter-13000 loss: 0.9838
=========================================================================
te-Jamanges mert. Ssiceuntr theas Ogeves lakke pstreutian Japan bas meity nu hIn -JaDgest lyUpe iggt
=========================================================================


Out[129]:
<__main__.RNN at 0x7f00f78b9630>

In [5]:
x= []
for i in range(5):
    for j in range(3):
        x.append

In [9]:
class initTest:    
    def __init__(self, D, C, H, depth):
        self.models = []
        for layer in range(depth):
            model = dict(
                Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
                Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
                Why=np.random.randn(H, D) / np.sqrt(C / 2.),
                bh=np.zeros((1, H)),
                by=np.zeros((1, D))
            )
            self.models.append(model)

In [10]:
test = initTest(C=vocab_size, D=vocab_size, depth=2, H=H)
test.models[0].items()
test.models[0]['by']
# for item in test.models[1]:
#     print(item)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.]])